In [1]:
from google.colab import drive
drive.mount('/content/content')

Mounted at /content/content


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/content/content/MyDrive/Colab Notebooks/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.shape

(7043, 21)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [7]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


Perform initial data preparation by converting the 'TotalCharges' column to numeric values and filling missing values with 0.

In [9]:
# missing values in the  columns
missing_values = df.isnull().sum()
print(f"Number of missing values in df: {missing_values}")

# Convert 'TotalCharges' to numeric, and fill missing values with 0
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.fillna(0, inplace=True)

# Check if the conversion and filling were successful
missing_values_after = df.isnull().sum()
print(f"Number of missing values in the data after filling: {missing_values_after}")

Number of missing values in df: customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64
Number of missing values in the data after filling: customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
C

In [10]:
# Step 2: Convert the 'Churn' column to binary values
df['Churn'] = df['Churn'].map({'No': 0, 'Yes': 1})

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Step 3: Select specific features
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
                       'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                       'Contract', 'PaperlessBilling', 'PaymentMethod']

numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Step 4: Preprocess the data
# Scale the numerical features using StandardScaler
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Perform one-hot encoding on categorical features using pandas
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)


In [16]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split

# Split the dataset into features and target variable (Churn)
X = df.drop(columns=['Churn','customerID'])
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [17]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the models
random_forest = RandomForestClassifier(random_state=1)
extra_trees = ExtraTreesClassifier(random_state=1)
xgboost = XGBClassifier(random_state=1)
lightgbm = LGBMClassifier(random_state=1)

random_forest.fit(X_train, y_train)
extra_trees.fit(X_train, y_train)
xgboost.fit(X_train, y_train)
lightgbm.fit(X_train, y_train)

# Make predictions
rf_predictions = random_forest.predict(X_test)
et_predictions = extra_trees.predict(X_test)
xgb_predictions = xgboost.predict(X_test)
lgbm_predictions = lightgbm.predict(X_test)

# Evaluate the models
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
lgbm_accuracy = accuracy_score(y_test, lgbm_predictions)

# Generate classification reports
rf_report = classification_report(y_test, rf_predictions)
et_report = classification_report(y_test, et_predictions)
xgb_report = classification_report(y_test, xgb_predictions)
lgbm_report = classification_report(y_test, lgbm_predictions)

print("Random Forest Classifier Accuracy: {:.2f}%".format(rf_accuracy * 100))
print("Random Forest Classifier Classification Report:\n", rf_report)

print("\nExtra Trees Classifier Accuracy: {:.2f}%".format(et_accuracy * 100))
print("Extra Trees Classifier Classification Report:\n", et_report)

print("\nXGBoost Classifier Accuracy: {:.2f}%".format(xgb_accuracy * 100))
print("XGBoost Classifier Classification Report:\n", xgb_report)

print("\nLightGBM Classifier Accuracy: {:.2f}%".format(lgbm_accuracy * 100))
print("LightGBM Classifier Classification Report:\n", lgbm_report)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000961 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 638
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785
Random Forest Classifier Accuracy: 80.20%
Random Forest Classifier Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.88      0.87      1061
           1       0.61      0.55      0.58       348

    accuracy                           0.80      1409
   macro avg       0.73      0.72      0.72      1409
wei

In [19]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV

# Split the dataset into features and target variable (Churn)
X = df.drop(columns=['Churn'])
y = df['Churn']

# Define the hyperparameter grid
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'max_features': max_features
}

# Initialize the Extra Trees Classifier
extra_trees = ExtraTreesClassifier(random_state=1)

# Initialize the Randomized Search
random_search = RandomizedSearchCV(
    estimator=extra_trees,
    param_distributions=hyperparameter_grid,
    n_iter=10,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=1
)

# Perform the Randomized Search
random_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:")
print(random_search.best_params_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters:
{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}


In [23]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier


# Initialize the Extra Trees Classifier with the best hyperparameters
best_hyperparameters = {
    'n_estimators': 1000,
    'min_samples_split': 2,
    'min_samples_leaf': 8,
    'max_features': None,
    'random_state': 1
}

extra_trees_best = ExtraTreesClassifier(**best_hyperparameters)

# Train the model
extra_trees_best.fit(X_train, y_train)

# Get feature importances
feature_importances = extra_trees_best.feature_importances_

# Create a DataFrame to display feature names and their importance scores
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Get the two most important features
top_two_features = importance_df.head(2)

print("Top Two Most Important Features:")
print(top_two_features)



Top Two Most Important Features:
                        Feature  Importance
0                        tenure    0.246831
10  InternetService_Fiber optic    0.219035
